In [11]:
'''
This notebook should stay in root directory
'''
# this notebook should stay in root
import os
import torch
import numpy as np
import pickle
import time
import random
from generation_manager import GeneratorManager

In [2]:
curr_model_path = 'ckpt/ec_3_2_1_4_warmup_1000_earlystop_015_flip_LR01_2batch.pth' # model checkpoint
# Saving only state_dict:
#torch.save(model.state_dict(), model_state_dict_path)
#model_state_dict_path = load_model_path + 'model_only_state_dict_v0Last.pth'
#print('Model state dict saved')

In [3]:
def save_sequences(predicted, tokens_probs_all, name):
    data_dir = "data_generation/3.2.1.4/"
    query = name
    os.makedirs(data_dir, exist_ok=True)
    
    predicted_data_file = os.path.join(data_dir, "predicted_data_" + query + ".p")
    with open(predicted_data_file, "wb") as file:
        pickle.dump(predicted, file)

    tokens_probs_all_file = os.path.join(data_dir, "tokens_prob_data_" + query + ".p")
    with open(tokens_probs_all_file, "wb") as file:
        pickle.dump(tokens_probs_all, file)
        

In [4]:
# GENERATION parameters
temperature = 0.9
penalty = 1.2
top_p = 0.5
np.random.seed(1337)
torch.manual_seed(1337)
random.seed(1337)
torch.cuda.manual_seed_all(1337)

generator = GeneratorManager(curr_model_path, penalty=penalty, top_p=top_p, temperature=temperature)

MODEL SIZE: 
1280
Found PyTorch checkpoint at  ckpt/ec_3_2_1_4_warmup_1000_earlystop_015_flip_LR01_2batch.pth
GPU aviable. Previous checkpoint loaded in GPU


In [5]:

def generation(name, keys):
    predicted = {}
    # true_tokens_index_in_probs_all = []
    tokens_probs_all = []
    # true_value = []
    seq_number = 0
    #for entry in random_selection:
    while seq_number <= 1000:
        # input_seq = entry['sequence']['metadata']['sequence']
        # FOR TESTIG the code:
        # input_seq = input_seq[0:24]
        input_seq = ''
        
        # if no tax keys:
        tax_lineage = keys
        offset = 0
        
        #print('generating sequences...')
        #start_time = time.time()
        res, stop_prob_one_sequence, tokens_prob = generator.generation_complete_sequence_new(input_seq, tax_lineage, offset)
        #end_time = time.time()
        #elapsed_time = end_time - start_time
        #print(f"Done. Time taken: {elapsed_time} seconds.")
        
        tokens_probs_all.append(tokens_prob)
        
        #print("Input: ", input_seq)
        #print("Res: ", res)
        #print('stop probs: ', stop_prob_one_sequence)
        #print('len stop probs: ', len(stop_prob_one_sequence))
        #print('len res: ', len(res))
        # true_value.append(input_seq)
        # true_tokens_index_in_prob = []
        # true_tokens_index_in_probs_all.append(true_tokens_index_in_prob)
        # print('input_offset should be nothing: ', input_seq[:offset])
        
        id = 'sequence_'+str(seq_number)
        predicted[id] = {'seq':input_seq[:offset] + res, 'stop_probs':stop_prob_one_sequence}
        
        seq_number += 1

        if (seq_number%20) == 0:
            print('hello, this is seq_number', seq_number)
            print('hello, this is seq_number module: ', seq_number%10)
            print('the model has generated: ', seq_number , ' sequences')
            print('saving sequences')
            save_sequences(predicted, tokens_probs_all, name)
    print('GENERATION ENDED')


In [ ]:
generation('p0_5_percent_key_0', [0])

In [ ]:
#generation('p0_5_percent_penalty_0_key_0_1', [0, 1])

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# GENERATION parameters
temperature = 0.9
penalty = 1.2
top_p = 0.75
np.random.seed(1337)
torch.manual_seed(1337)
random.seed(1337)
torch.cuda.manual_seed_all(1337)

generator = GeneratorManager(curr_model_path, penalty=penalty, top_p=top_p, temperature=temperature)

In [ ]:
generation('p0_75_percent_key_0', [0])

In [ ]:
torch.cuda.empty_cache()